In [2]:
# Import Required Libraries
import requests
import re
from bs4 import BeautifulSoup

In [3]:
# Request all deaths for those aged 0 - 5 for the year 2000
page = requests.get('http://homicide.latimes.com/age/0/year/2000')

In [4]:
# Parse the webpage and make the output pretty
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script type="text/javascript">
   var _sf_startpt=(new Date()).getTime()
  </script>
  <!-- METADATA -->
  <title>
   Victims in 2000 , ages 0-5 - Los Angeles Murders and Homicides - The Homicide Report - Los Angeles Times
  </title>
  <meta charset="utf-8"/>
  <meta content="Los Angeles County murder and homicides victims, and their stories." name="description"/>
  <meta content="murders, homicides, victims, violence, los angeles, data desk, los angeles times, latimes, l.a. times" name="keywords"/>
  <meta content="width=device-width,  initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <link href="//d1qqc1e9kvmdh8.cloudfront.net/img/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://plus.google.com/u/0/112727226361732924638" rel="publisher">
   <meta content="a9pAUAoFTyj7Nut3H5OiQ-fUyyqdwdVT7VcFGICrpTo" name="google-site-verification">
    <meta content="The Homicide Repor

In [5]:
# Notice that the data we need is collected under the 'article' tag
# We can see that the class is "post-teaser row"

child_name_list = soup.find_all('article')
child_name_list

[<article class="post-teaser row">
 <aside class="span2 offset1 hidden-phone">
 <img class="post-list-thumb" src="http://dev.virtualearth.net/REST/v1/Imagery/Map/Road/34.0549773,-118.2464487/12?mapSize=135%2C135&amp;key=AqEQ_HqipMZe_KnMeHEJ_CEtkNG9Y34_aXaGeIya4fBtc4hTIA9KYzMfFaK5nbK5&amp;pushpin=34.0549773%2C-118.2464487%3B88&amp;format=jpeg"/>
 <div class="post-list-badge">
         Died on
         <div class="death-date">Dec. 20</div>
 <ul class="badge-location">
 <li><a href="/neighborhood/downtown/year/all">Downtown</a></li>
 <li>111 N. Hill St.</li>
 </ul>
 </div>
 </aside>
 <article class="span8 post-list-content">
 <hgroup>
 <h2><a href="/post/joan-angelique-richards/">Joan Angelique Richards, 5
 </a></h2>
 <p class="post-date">Posted <time datetime="">Dec. 20, 2000</time></p>
 </hgroup>
 <div class="body">
 <p> Joan Angelique Richards, a 5-year-old <a href="/race/black/year/all">black</a> <a href="/gender/female/year/all">female</a>, died Wednesday, Dec. 20, after sustaining <

In [6]:
# Let's collect all the records and see how many we get
results1 = soup.find_all('article', attrs={'class':'post-teaser row'})

In [7]:
len(results1 ) 

15

In [8]:
# Print a managable subset of the data (4 records)
results1[0:3] 

[<article class="post-teaser row">
 <aside class="span2 offset1 hidden-phone">
 <img class="post-list-thumb" src="http://dev.virtualearth.net/REST/v1/Imagery/Map/Road/34.0549773,-118.2464487/12?mapSize=135%2C135&amp;key=AqEQ_HqipMZe_KnMeHEJ_CEtkNG9Y34_aXaGeIya4fBtc4hTIA9KYzMfFaK5nbK5&amp;pushpin=34.0549773%2C-118.2464487%3B88&amp;format=jpeg"/>
 <div class="post-list-badge">
         Died on
         <div class="death-date">Dec. 20</div>
 <ul class="badge-location">
 <li><a href="/neighborhood/downtown/year/all">Downtown</a></li>
 <li>111 N. Hill St.</li>
 </ul>
 </div>
 </aside>
 <article class="span8 post-list-content">
 <hgroup>
 <h2><a href="/post/joan-angelique-richards/">Joan Angelique Richards, 5
 </a></h2>
 <p class="post-date">Posted <time datetime="">Dec. 20, 2000</time></p>
 </hgroup>
 <div class="body">
 <p> Joan Angelique Richards, a 5-year-old <a href="/race/black/year/all">black</a> <a href="/gender/female/year/all">female</a>, died Wednesday, Dec. 20, after sustaining <

In [9]:
# Let's look at the first record to see what needs to be grabbed
first_result = results1[0] 
first_result  

<article class="post-teaser row">
<aside class="span2 offset1 hidden-phone">
<img class="post-list-thumb" src="http://dev.virtualearth.net/REST/v1/Imagery/Map/Road/34.0549773,-118.2464487/12?mapSize=135%2C135&amp;key=AqEQ_HqipMZe_KnMeHEJ_CEtkNG9Y34_aXaGeIya4fBtc4hTIA9KYzMfFaK5nbK5&amp;pushpin=34.0549773%2C-118.2464487%3B88&amp;format=jpeg"/>
<div class="post-list-badge">
        Died on
        <div class="death-date">Dec. 20</div>
<ul class="badge-location">
<li><a href="/neighborhood/downtown/year/all">Downtown</a></li>
<li>111 N. Hill St.</li>
</ul>
</div>
</aside>
<article class="span8 post-list-content">
<hgroup>
<h2><a href="/post/joan-angelique-richards/">Joan Angelique Richards, 5
</a></h2>
<p class="post-date">Posted <time datetime="">Dec. 20, 2000</time></p>
</hgroup>
<div class="body">
<p> Joan Angelique Richards, a 5-year-old <a href="/race/black/year/all">black</a> <a href="/gender/female/year/all">female</a>, died Wednesday, Dec. 20, after sustaining <a href="/cause/blunt

In [10]:
# This is a tricky one, we need to grab only the spatial coordinates, luckily there's a comma in between them
images = first_result.findAll('img', class_="post-list-thumb")[0]['src']
images

'http://dev.virtualearth.net/REST/v1/Imagery/Map/Road/34.0549773,-118.2464487/12?mapSize=135%2C135&key=AqEQ_HqipMZe_KnMeHEJ_CEtkNG9Y34_aXaGeIya4fBtc4hTIA9KYzMfFaK5nbK5&pushpin=34.0549773%2C-118.2464487%3B88&format=jpeg'

In [11]:
# Take the 8th pience of information after finding instances of the "/"
images.split("/")[8]

# Find all img tags, split the data and get the spatial coordinates
first_result.findAll('img', class_="post-list-thumb")[0]['src'].split("/")[8]

'34.0549773,-118.2464487'

In [12]:
# It's a bit tedious but eventually we can find all the data we need
# This field yeilds the neighborhood where the death occurred
sec_result = results1[0] 
sec_result.find('li').text

'Downtown'

In [13]:
# Here we have the date of the child's death
first_result.find('div', 'death-date').text

'Dec. 20'

In [21]:
# This is the child's gender
first_result.findAll('a')[1].text.strip()

'Joan Angelique Richards, 5'

In [15]:
# And we have a short description of the death, before a longer synopsis is given
first_result.findAll('p')[1].text.strip()

"Joan Angelique Richards, a 5-year-old black female, died Wednesday, Dec. 20, after sustaining blunt force trauma in Downtown, according to Los Angeles County coroner's records."

In [16]:
results1[0]

<article class="post-teaser row">
<aside class="span2 offset1 hidden-phone">
<img class="post-list-thumb" src="http://dev.virtualearth.net/REST/v1/Imagery/Map/Road/34.0549773,-118.2464487/12?mapSize=135%2C135&amp;key=AqEQ_HqipMZe_KnMeHEJ_CEtkNG9Y34_aXaGeIya4fBtc4hTIA9KYzMfFaK5nbK5&amp;pushpin=34.0549773%2C-118.2464487%3B88&amp;format=jpeg"/>
<div class="post-list-badge">
        Died on
        <div class="death-date">Dec. 20</div>
<ul class="badge-location">
<li><a href="/neighborhood/downtown/year/all">Downtown</a></li>
<li>111 N. Hill St.</li>
</ul>
</div>
</aside>
<article class="span8 post-list-content">
<hgroup>
<h2><a href="/post/joan-angelique-richards/">Joan Angelique Richards, 5
</a></h2>
<p class="post-date">Posted <time datetime="">Dec. 20, 2000</time></p>
</hgroup>
<div class="body">
<p> Joan Angelique Richards, a 5-year-old <a href="/race/black/year/all">black</a> <a href="/gender/female/year/all">female</a>, died Wednesday, Dec. 20, after sustaining <a href="/cause/blunt

In [20]:
# Now we loop through the records and store each one into a field, here I am looping over the first three
records = []  
for result in results1[0:3]:  
    neighborhood = result.find('li').text 
    death_date = result.find('div', 'death-date').text.strip()
    name = result.findAll('a')[1].text.strip()
    race = result.findAll('a')[2].text.strip()
    gender = result.findAll('a')[3].text.strip()
    coords = result.findAll('img', class_="post-list-thumb")[0]['src'].split("/")[8]
    year = 2000
    descript = result.findAll('p')[1].text.strip()
    records.append((neighborhood, death_date, name, race, gender, coords, year, descript))
records
 

[('Downtown',
  'Dec. 20',
  'Joan Angelique Richards, 5',
  'black',
  'female',
  '34.0549773,-118.2464487',
  2000,
  "Joan Angelique Richards, a 5-year-old black female, died Wednesday, Dec. 20, after sustaining blunt force trauma in Downtown, according to Los Angeles County coroner's records."),
 ('Chesterfield Square',
  'Dec. 10',
  'Aaliyah Oneil',
  'black',
  'female',
  '33.980837,-118.311459',
  2000,
  "Aaliyah Oneil, a black female, died Sunday, Dec. 10, after sustaining blunt force trauma in Chesterfield Square, according to Los Angeles County coroner's records."),
 ('Rancho Palos Verdes',
  'Nov. 8',
  'Lauren Serene Key-Marer, 4',
  'white',
  'female',
  '33.7534451,-118.326726',
  2000,
  "Lauren Serene Key-Marer, a 4-year-old white female, died Wednesday, Nov. 8, in Rancho Palos Verdes, according to Los Angeles County coroner's records.")]

In [18]:
#Clearly that format is not managable so we will convert the data into a dataframe and can save as a csv file
import pandas as pd  
df = pd.DataFrame(records, columns=['neighborhood', 'death_date', 'name', 'race',  'gender', 'coords', 'year', 'descript'])  
df

,neighborhood,death_date,name,race,gender,coords,year,descript
0,Downtown,Dec. 20,"Joan Angelique Richards, 5",black,female,"34.0549773,-118.2464487",2000,"Joan Angelique Richards, a 5-year-old black fe..."
1,Chesterfield Square,Dec. 10,Aaliyah Oneil,black,female,"33.980837,-118.311459",2000,"Aaliyah Oneil, a black female, died Sunday, De..."
2,Rancho Palos Verdes,Nov. 8,"Lauren Serene Key-Marer, 4",white,female,"33.7534451,-118.326726",2000,"Lauren Serene Key-Marer, a 4-year-old white fe..."
